# Pyterrier Notebook for evaluating project test runs

### Install dependencies 

In [ ]:
!pip3 install python-terrier tira==0.0.88 ir_datasets

Load imports and datasets

In [ ]:
from trectools import TrecRun, TrecQrel, TrecEval
from tira.rest_api_client import Client
from glob import glob
tira = Client()


def load_qrels(dataset):
    return TrecQrel(tira.download_dataset('ir-lab-jena-leipzig-wise-2023', dataset, truth_dataset=True) + '/qrels.txt')

training_qrels = load_qrels('training-20231104-training')
validation_qrels = load_qrels('validation-20231104-training')

Define function to read runs from from TREC run file

In [3]:
def evaluate_run(run, qrels):
    run = TrecRun(run)
    trec_eval = TrecEval(run, qrels)

    return {
        'run': run.get_runid(),
        'nDCG@10': trec_eval.get_ndcg(depth=10),
        'nDCG@10 (unjudgedRemoved)': trec_eval.get_ndcg(depth=10, removeUnjudged=True),
        'MAP': trec_eval.get_map(depth=10),
        'MRR': trec_eval.get_reciprocal_rank()
    }

Convert to pandas dataframe and sort by ndcg at 10

In [2]:
import pandas as pd

df = []
for r in glob('runs_milestone3/*.txt'):
    df += [evaluate_run(r, training_qrels)]
df = pd.DataFrame(df)
df.sort_values('nDCG@10', ascending=False)

NameError: name 'glob' is not defined